# Flight Price Forecast - Feature Selection

This notebook focuses on feature engineering and selection to identify the most important factors affecting airline pricing:

- Create new features from existing data (seasonality, temporal features, etc.)
- Apply tree-based models to determine feature importance
- Use various feature selection techniques
- Select optimal feature set for modeling

**Prerequisites**: Complete `01_data_preprocessing.ipynb` and `02_data_analysis.ipynb`.


## 1. Load and Prepare Data


## 2. Feature Engineering


## 3. Tree-Based Feature Importance


## 4. Statistical Feature Selection


## 5. Final Feature Selection and Export
